In [ ]:
#@title Define Battery Parameters

energy_capacity = 100 #@param {type:"number"}
#@markdown > The maximum volume of energy that can be stored in the battery system, measured in MWh. Note that this is the quantity of energy stored in the system after charging efficiency losses and before discharging efficiency losses.
charge_power_limit = 50 #@param {type:"number"}
#@markdown > The maximum power rate at which the battery can charge, measured in MW.
discharge_power_limit = 50 #@param {type:"number"}
#@markdown >  The maximum power rate at which the battery can dicharge, measured in MW.
charge_efficiency = 0.95 #@param {type:"number"}
#@markdown > The efficiency at which energy can enter the battery. For example, charging at 1 MW for 1 hour with a 95% charge efficiency will result in 0.95 MWh of energy stored in the battery.
discharge_efficiency = 0.95 #@param {type:"number"}
#@markdown > The efficiency at which energy can leave the battery. For example, discharging at 1 MW for 1 hour with a 95% discharge efficiency will result in 0.95 MWh of energy to the grid.
SOC_max = 100 #@param {type:"number"}
#@markdown > The maximum allowable amount of energy that can be stored in the battery, measured in MWh. There can be longevity or other reasons for making this less than the Energy Capacity.
SOC_min = 10 #@param {type:"number"}
#@markdown > The minimum allowable amount of energy that can be stored in the battery, measured in MWh. There can be longevity or other reasons for making this greater than zero.
daily_cycle_limit = 1 #@param {type:"number"}
#@markdown > The maximum number of cycles allowed in a day. This constraint can be imposed for battery health reasons.
annual_cycle_limit = 400 #@param {type:"number"}
#@markdown > The maximum number of cycles allowed in a year. This constraint can be imposed for battery warranty or long-term degradation limiting reasons.
SOC_initial = SOC_min
#@markdown > The SOC of the first interval of the analysis. For analyses of short periods of time, this can have a meaningful impact on net revenue. For analyses over a month or so, it doesn't matter much.


In [ ]:
import requests
import pandas as pd
from io import BytesIO

# Google Drive direct download link for the file ID
file_id = '181pas7UfRPTCUWqNMFFIJX1uWshOLUPF'
url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # If the request was successful, read the downloaded bytes into a pandas DataFrame
    excel_data = BytesIO(response.content)
    df = pd.read_excel(excel_data, sheet_name=0)  # Use the correct sheet index or name

# The format appears to be missing the year, so we'll include it and handle errors without dropping rows
year_to_add = "2024"  # Example year to add for conversion, needs to be leap year as data includes Feb 29th values
df['Day, time'] = pd.to_datetime(year_to_add + df['Day, time'], errors='coerce', format='%Y-%m-%d %H:%M:%S')

# Rename the columns to match the expected names for the analysis
df.rename(columns={
    'Day, time': 'interval_start_local',
    'Average Wholesale Spot price 2018-2022': 'lmp',
    'MEC BESS MW (64MWac connection)': 'max_export_capacity'
}, inplace=True)

# Extract the 'lmp' and 'max_export_capacity' columns
da_prices = df['lmp'].tolist()
max_export_capacity = df['max_export_capacity'].tolist()

 # Print the first 20 lines of the dataframe to verify
print(df.head(20))


   interval_start_local      lmp  Solar 43.6 adjusted  max_export_capacity
0   2024-01-01 00:00:00  42.7800           -45.520315            64.045520
1   2024-01-01 01:00:00  34.5475           -45.520315            64.045520
2   2024-01-01 02:00:00  30.0925           -45.520315            64.045520
3   2024-01-01 03:00:00  27.6975           -45.520315            64.045520
4   2024-01-01 04:00:00  25.3750           -45.520315            64.045520
5   2024-01-01 05:00:00  24.3725           -45.520315            64.045520
6   2024-01-01 06:00:00  23.2850           -45.520315            64.045520
7   2024-01-01 07:00:00  24.5700           -45.520315            64.045520
8   2024-01-01 08:00:00  29.1625           690.603257            63.309397
9   2024-01-01 09:00:00  34.8125          1624.090027            62.375910
10  2024-01-01 10:00:00  36.7825          2072.455993            61.927544
11  2024-01-01 11:00:00  40.5225          2996.073925            61.003926
12  2024-01-01 12:00:00  

In [ ]:
#@title Standalone Dispatch

!pip install pulp
from pulp import LpVariable, LpProblem, LpMaximize, lpSum, value, LpStatus

# Assuming da_prices has been defined in the data preparation code

# Price Forecast for num_intervals intervals
num_intervals = len(da_prices)
print(f"Number of intervals: {num_intervals}")

num_days = num_intervals / 24  # Adjusted for hourly data
print(f"Number of days: {num_days}")

total_cycle_limit = (num_days / 365) * annual_cycle_limit
print(f"Total cycle limit: {total_cycle_limit}")

# Variables
charge_vars = LpVariable.dicts("Charging", range(num_intervals), lowBound=0, upBound=charge_power_limit)
discharge_vars = LpVariable.dicts("Discharging", range(num_intervals), lowBound=0, upBound=discharge_power_limit)
SOC_vars = LpVariable.dicts("SOC", range(num_intervals+1), lowBound=SOC_min, upBound=SOC_max)  # Including initial SOC

# Problem
prob = LpProblem("Battery Scheduling", LpMaximize)

# Objective function
prob += lpSum([da_prices[t]*discharge_efficiency*discharge_vars[t] - da_prices[t]*charge_vars[t]/charge_efficiency for t in range(num_intervals)])

# Constraints
# Initial SOC constraint
prob += SOC_vars[0] == SOC_initial

# SOC update constraints
for t in range(num_intervals):
    prob += SOC_vars[t+1] == SOC_vars[t] + charge_efficiency * charge_vars[t] - discharge_efficiency * discharge_vars[t]


# Charge/Discharge constraints based on SOC
for t in range(num_intervals):
    prob += SOC_vars[t] + charge_efficiency*charge_vars[t] <= SOC_max  # Cannot charge if SOC_max is reached
    prob += SOC_vars[t] - discharge_vars[t]*discharge_efficiency >= SOC_min  # Cannot discharge below SOC_min

# Simultaneous charge and discharge constraint
for t in range(num_intervals):
    prob += charge_vars[t] + discharge_vars[t] <= max(charge_power_limit, discharge_power_limit)

# Daily cycle limit constraint
for day_start in range(0, num_intervals, 24):
    day_end = min(day_start + 24, num_intervals)
    prob += lpSum([charge_vars[t] for t in range(day_start, day_end)]) * charge_efficiency / energy_capacity <= daily_cycle_limit

# Annual cycle limit constraint
prob += lpSum([charge_vars[t] for t in range(num_intervals)]) * charge_efficiency / energy_capacity <= total_cycle_limit

# Solve the problem
prob.solve()

# Check the status of the solution
print("Status:", LpStatus[prob.status])

# Assuming the problem is solved successfully, the rest of the code can proceed
# However, you should include checks to ensure the problem was solved correctly before proceeding

# Create the battery dispatch DataFrame
battery_dispatch_df = pd.DataFrame({
    'interval_start_local': df['interval_start_local'],  # Assuming 'df' is your prepped DataFrame
    'lmp': da_prices
})

# Create series from the optimization results for the unconstrained scenario
discharge_vars_series_unconstrained = pd.Series([value(discharge_vars[t]) if discharge_vars[t].varValue is not None else 0 for t in range(num_intervals)], name='discharge_vars')
charge_vars_series_unconstrained = pd.Series([value(charge_vars[t]) if charge_vars[t].varValue is not None else 0 for t in range(num_intervals)], name='charge_vars')
soc_vars_series_unconstrained = pd.Series([value(SOC_vars[t]) if SOC_vars[t].varValue is not None else 0 for t in range(num_intervals+1)], name='soc_vars')[:-1]

# Create a DataFrame for the unconstrained scenario
battery_dispatch_df_unconstrained = pd.DataFrame({
    'discharge_vars': discharge_vars_series_unconstrained.values.round(1),
    'charge_vars': charge_vars_series_unconstrained.values.round(1),
    'SOC_vars': soc_vars_series_unconstrained.values.round(1),
    'lmp': da_prices
})

# Set the index to 'interval_start_local'
battery_dispatch_df_unconstrained.set_index(df['interval_start_local'], inplace=True)


# Display the first few rows of the dispatch schedule
print(battery_dispatch_df.head(24))

# Check the problem status and results
print("Status:", LpStatus[prob.status])



Number of intervals: 8760
Number of days: 365.0
Total cycle limit: 400.0


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning:

Spaces are not permitted in the name. Converted to '_'



Status: Optimal
   interval_start_local      lmp
0   2024-01-01 00:00:00  42.7800
1   2024-01-01 01:00:00  34.5475
2   2024-01-01 02:00:00  30.0925
3   2024-01-01 03:00:00  27.6975
4   2024-01-01 04:00:00  25.3750
5   2024-01-01 05:00:00  24.3725
6   2024-01-01 06:00:00  23.2850
7   2024-01-01 07:00:00  24.5700
8   2024-01-01 08:00:00  29.1625
9   2024-01-01 09:00:00  34.8125
10  2024-01-01 10:00:00  36.7825
11  2024-01-01 11:00:00  40.5225
12  2024-01-01 12:00:00  44.1575
13  2024-01-01 13:00:00  43.0775
14  2024-01-01 14:00:00  43.1625
15  2024-01-01 15:00:00  44.9175
16  2024-01-01 16:00:00  54.3850
17  2024-01-01 17:00:00  64.9225
18  2024-01-01 18:00:00  62.7650
19  2024-01-01 19:00:00  59.7050
20  2024-01-01 20:00:00  53.9375
21  2024-01-01 21:00:00  43.0550
22  2024-01-01 22:00:00  46.5775
23  2024-01-01 23:00:00  35.2000
Status: Optimal


In [ ]:
#@title Solar Constrained Dispatch

!pip install pulp
from pulp import LpVariable, LpProblem, LpMaximize, lpSum, value, LpStatus

# Assuming da_prices has been defined in the data preparation code

# Price Forecast for num_intervals intervals
num_intervals = len(da_prices)
print(f"Number of intervals: {num_intervals}")

num_days = num_intervals / 24  # Adjusted for hourly data
print(f"Number of days: {num_days}")

total_cycle_limit = (num_days / 365) * annual_cycle_limit
print(f"Total cycle limit: {total_cycle_limit}")

# Variables
charge_vars = LpVariable.dicts("Charging", range(num_intervals), lowBound=0, upBound=charge_power_limit)
discharge_vars = LpVariable.dicts("Discharging", range(num_intervals), lowBound=0, upBound=discharge_power_limit)
SOC_vars = LpVariable.dicts("SOC", range(num_intervals+1), lowBound=SOC_min, upBound=SOC_max)  # Including initial SOC

# Problem
prob = LpProblem("Battery Scheduling", LpMaximize)

# Objective function
prob += lpSum([da_prices[t]*discharge_efficiency*discharge_vars[t] - da_prices[t]*charge_vars[t]/charge_efficiency for t in range(num_intervals)])

# Constraints
# Initial SOC constraint
prob += SOC_vars[0] == SOC_initial

# SOC update constraints
for t in range(num_intervals):
    prob += SOC_vars[t+1] == SOC_vars[t] + charge_efficiency * charge_vars[t] - discharge_efficiency * discharge_vars[t]


# Charge/Discharge constraints based on SOC
for t in range(num_intervals):
    prob += SOC_vars[t] + charge_efficiency*charge_vars[t] <= SOC_max  # Cannot charge if SOC_max is reached
    prob += SOC_vars[t] - discharge_vars[t]*discharge_efficiency >= SOC_min  # Cannot discharge below SOC_min

# Simultaneous charge and discharge constraint
for t in range(num_intervals):
    prob += charge_vars[t] + discharge_vars[t] <= max(charge_power_limit, discharge_power_limit)

# Daily cycle limit constraint
for day_start in range(0, num_intervals, 24):
    day_end = min(day_start + 24, num_intervals)
    prob += lpSum([charge_vars[t] for t in range(day_start, day_end)]) * charge_efficiency / energy_capacity <= daily_cycle_limit

# Annual cycle limit constraint
prob += lpSum([charge_vars[t] for t in range(num_intervals)]) * charge_efficiency / energy_capacity <= total_cycle_limit

# Modify the Constraints to Include max_export_capacity
for t in range(num_intervals):
    prob += discharge_vars[t] <= max_export_capacity[t]  # Discharge limited by export capacity

# Solve the problem
prob.solve()

# Check the status of the solution
print("Status:", LpStatus[prob.status])

# Assuming the problem is solved successfully, the rest of the code can proceed
# However, you should include checks to ensure the problem was solved correctly before proceeding

# Create the battery dispatch DataFrame
battery_dispatch_df = pd.DataFrame({
    'interval_start_local': df['interval_start_local'],  # Assuming 'df' is your prepped DataFrame
    'lmp': da_prices
})

# Create series from the optimization results for the constrained scenario
discharge_vars_series_constrained = pd.Series([value(discharge_vars[t]) if discharge_vars[t].varValue is not None else 0 for t in range(num_intervals)], name='discharge_vars')
charge_vars_series_constrained = pd.Series([value(charge_vars[t]) if charge_vars[t].varValue is not None else 0 for t in range(num_intervals)], name='charge_vars')
soc_vars_series_constrained = pd.Series([value(SOC_vars[t]) if SOC_vars[t].varValue is not None else 0 for t in range(num_intervals+1)], name='soc_vars')[:-1]

# Create a DataFrame for the constrained scenario
battery_dispatch_df_constrained = pd.DataFrame({
    'discharge_vars': discharge_vars_series_constrained.values.round(1),
    'charge_vars': charge_vars_series_constrained.values.round(1),
    'SOC_vars': soc_vars_series_constrained.values.round(1),
    'lmp': da_prices
})

# Set the index to 'interval_start_local'
battery_dispatch_df_constrained.set_index(df['interval_start_local'], inplace=True)

# Display the first few rows of the dispatch schedule
print(battery_dispatch_df.head(24))

# Check the problem status and results
print("Status:", LpStatus[prob.status])

Number of intervals: 8760
Number of days: 365.0
Total cycle limit: 400.0


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning:

Spaces are not permitted in the name. Converted to '_'



Status: Optimal
   interval_start_local      lmp
0   2024-01-01 00:00:00  42.7800
1   2024-01-01 01:00:00  34.5475
2   2024-01-01 02:00:00  30.0925
3   2024-01-01 03:00:00  27.6975
4   2024-01-01 04:00:00  25.3750
5   2024-01-01 05:00:00  24.3725
6   2024-01-01 06:00:00  23.2850
7   2024-01-01 07:00:00  24.5700
8   2024-01-01 08:00:00  29.1625
9   2024-01-01 09:00:00  34.8125
10  2024-01-01 10:00:00  36.7825
11  2024-01-01 11:00:00  40.5225
12  2024-01-01 12:00:00  44.1575
13  2024-01-01 13:00:00  43.0775
14  2024-01-01 14:00:00  43.1625
15  2024-01-01 15:00:00  44.9175
16  2024-01-01 16:00:00  54.3850
17  2024-01-01 17:00:00  64.9225
18  2024-01-01 18:00:00  62.7650
19  2024-01-01 19:00:00  59.7050
20  2024-01-01 20:00:00  53.9375
21  2024-01-01 21:00:00  43.0550
22  2024-01-01 22:00:00  46.5775
23  2024-01-01 23:00:00  35.2000
Status: Optimal


In [ ]:
#@title Compare Results

import plotly.graph_objects as go

# Define a function to calculate the monthly metrics
def calculate_monthly_metrics(df):
    df['hourly_discharging_revenue'] = df['discharge_vars'] * df['lmp'] * discharge_efficiency
    df['hourly_charging_costs'] = df['charge_vars'] * df['lmp'] / charge_efficiency
    df['hourly_net_revenue'] = df['hourly_discharging_revenue'] - df['hourly_charging_costs']

    monthly_metrics = df[['hourly_discharging_revenue', 'hourly_charging_costs', 'hourly_net_revenue']].resample('MS').sum()
    monthly_metrics['Net Revenue ($)'] = monthly_metrics['hourly_net_revenue']

    return monthly_metrics[['Net Revenue ($)']]

# Calculate monthly metrics for both scenarios
monthly_metrics_unconstrained = calculate_monthly_metrics(battery_dispatch_df_unconstrained)
monthly_metrics_constrained = calculate_monthly_metrics(battery_dispatch_df_constrained)

# Combine metrics into a single DataFrame
comparison_df = pd.concat([monthly_metrics_unconstrained, monthly_metrics_constrained], axis=1)
comparison_df.columns = ['Net Revenue ($) Unconstrained', 'Net Revenue ($) Constrained']

# Calculate the percentage difference and add it to the comparison DataFrame
comparison_df['% Difference'] = ((comparison_df['Net Revenue ($) Unconstrained'] - comparison_df['Net Revenue ($) Constrained']) / comparison_df['Net Revenue ($) Unconstrained']) * 100

# Update the table generation code to include the new column
table = go.Figure(data=[go.Table(
    header=dict(values=comparison_df.columns.insert(0, 'Month'),
                fill_color='black',
                font=dict(color='white'),
                align='left'),
    cells=dict(values=[comparison_df.index.strftime('%Y-%m')] + [comparison_df[col].apply(lambda x: f"${x:,.0f}" if col.startswith('Net Revenue') else f"{x:.2f}%" ) for col in comparison_df.columns],
               fill_color='darkslategray',
               font=dict(color='white'),
               align='left'))
])

# Render the table
table.show()


# BAR CHART
total_revenue_unconstrained = monthly_metrics_unconstrained['Net Revenue ($)'].sum()
total_revenue_constrained = monthly_metrics_constrained['Net Revenue ($)'].sum()

# Calculate overall percentage difference
overall_percentage_difference = ((total_revenue_unconstrained - total_revenue_constrained) / total_revenue_unconstrained) * 100

# Create the bar chart
bar_chart = go.Figure(data=[
    go.Bar(name='Unconstrained', x=['Total Revenue'], y=[total_revenue_unconstrained]),
    go.Bar(name='Constrained', x=['Total Revenue'], y=[total_revenue_constrained])
])

# Update the layout for the bar chart
bar_chart.update_layout(
    title_text=f"Overall Revenue Comparison - {overall_percentage_difference:.2f}% Difference",
    template="plotly_dark",
    yaxis_title="Total Revenue ($)"
)

# Render the bar chart
bar_chart.show()


